In [44]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import onnxruntime as rt
import onnx
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import to_onnx
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils import resample

In [45]:
df = pd.read_csv('investigation_train_large_checked.csv')
data_description = pd.read_csv('data_description.csv', encoding='latin1')
target = df['checked']
features = df.drop(columns=['Ja', 'Nee','checked'])
features.head()

C:\Users\moegi\AppData\Local\Temp\ipykernel_20424\1069808925.py:1: DtypeWarning: Columns (317) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('investigation_train_large_checked.csv')


,adres_aantal_brp_adres,adres_aantal_verschillende_wijken,adres_aantal_verzendadres,adres_aantal_woonadres_handmatig,adres_dagen_op_adres,adres_recentst_onderdeel_rdam,adres_recentste_buurt_groot_ijsselmonde,adres_recentste_buurt_nieuwe_westen,adres_recentste_buurt_other,adres_recentste_buurt_oude_noorden,...,typering_dagen_som,typering_hist_aantal,typering_hist_inburgeringsbehoeftig,typering_hist_ind,typering_hist_sector_zorg,typering_ind,typering_indicatie_geheime_gegevens,typering_other,typering_transport__logistiek___tuinbouw,typering_zorg__schoonmaak___welzijn
0,1,1,0,0,23240,1,0,0,0,0,...,-644.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4,2,1,1,1971,1,0,0,1,0,...,3875.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
2,6,4,2,1,7247,0,0,0,1,0,...,3398.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
3,3,2,0,1,8060,1,0,0,1,0,...,128.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3,2,0,0,18705,1,0,0,0,0,...,2210.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0


In [46]:
checked_true = target.sum()
checked_false= len(target)-checked_true
print(len(target))
print('Checked == True: ',(checked_true/len(target))*100,'%')
print('Checked == False: ', (checked_false/len(target))*100,'%')
print(df.isnull().values.any())

3268
Checked == True:  15.758873929008569 %
Checked == False:  84.24112607099144 %
True


In [47]:
def get_protected_feature_indices():
    protected_indices = {
        'demographic': {
            'gender': [215],
            'age': [216],
            'language': [244,245,246,247]
        },
        'location': {
            'neighborhood': list(range(6,11)),
            'district': list(range(13,22)),
            'rotterdam': [11,12]
        },
        'family': {
            'children': list(range(282,289)),
            'partner': [302,303]
        },
        'health': {
            'physical': [54],
            'mental': [55],
            'medical': [67,68,69]
        },
        'financial': {
            'problems': [53,56],
            'income': [132,133,134]
        }
    }
    return protected_indices

def get_feature_importances():
    '''The data_description file assigns relative importance to each feature which can later be used to calculate risk_score of features.'''
    feature_importances = data_description['Relative importance']
    return feature_importances

def analyze_correlation_protected_features(data):
    '''For each protected feature, calculate the correlation and risk_score that other features have with it.'''
    protected_indices = get_protected_feature_indices()
    importances = get_feature_importances()
    results = {}

    for category, subcategories in protected_indices.items():
        results[category] = {}
        for subcategory, indices in subcategories.items():
            subcategory_results = []

            for idx in indices:
                protected_feature = data.columns[idx]

                for col_idx, col in enumerate(data.columns):
                    if col_idx == idx:
                        continue

                    correlation = data[protected_feature].corr(data[col], method='spearman')
                    importance = importances[col_idx]
                    risk_score = abs(correlation) * importance

                    if pd.notna(correlation):
                        subcategory_results.append({
                            'protected_feature': protected_feature,
                            'analyzed_feature': col,
                            'correlation': correlation,
                            'importance': importance,
                            'risk_score': risk_score
                        })

            subcategory_df = pd.DataFrame(subcategory_results)
            subcategory_df = subcategory_df.sort_values('risk_score',
                                                       ascending=False)

            results[category][subcategory] = subcategory_df

    return results

In [34]:
display_correlations = analyze_correlation_protected_features(features)

C:\Users\moegi\PycharmProjects\SWE-Testing4AI\.venv\Lib\site-packages\pandas\core\nanops.py:1632: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  return spearmanr(a, b)[0]


In [35]:
def display_correlation_results(results, top_n=5, correlation_threshold=0.5):
    for category, subcategories in results.items():
        print(f"\n{'='*80}")
        print(f"CATEGORY: {category.upper()}")
        print(f"{'='*80}")

        for subcategory, subcategory_df in subcategories.items():
            if isinstance(subcategory_df, pd.DataFrame) and not subcategory_df.empty:
                print(f"\nSUBCATEGORY: {subcategory}")
                print(f"{'-'*40}")

                # Display top N features by risk score
                print(f"\nTop {top_n} highest risk features:")
                top_features = subcategory_df.head(top_n)[
                    ['protected_feature', 'analyzed_feature', 'correlation',
                     'importance', 'risk_score']
                ]

                plt.figure(figsize=(10, 6))
                plt.gca().spines['top'].set_visible(False)
                bars = plt.bar(range(len(top_features)), top_features['risk_score'])

                plt.title(f'Top {top_n} Risk Scores for {category} - {subcategory}')
                plt.xlabel('Features')
                plt.ylabel('Risk Score')

                plt.xticks(range(len(top_features)),
                          top_features['analyzed_feature'],
                          rotation=45,
                          ha='right')

                for bar in bars:
                    height = bar.get_height()
                    plt.text(bar.get_x() + bar.get_width()/2., height,
                            f'{height:.2f}',
                            ha='center', va='bottom')

                plt.tight_layout()
                plt.show()

                # Display features with strong correlations
                strong_correlations = subcategory_df[
                    abs(subcategory_df['correlation']) >= correlation_threshold
                ].sort_values('correlation', ascending=False)

                print(f"\nSummary statistics:")
                print(f"- Total features analyzed: {len(subcategory_df)}")
                print(f"- Average absolute correlation: {subcategory_df['correlation'].abs().mean():.3f}")
                print(f"- Average risk score: {subcategory_df['risk_score'].mean():.3f}")
                print(f"- Number of strong correlations: {len(strong_correlations)}")
                print("\n")

In [36]:
def calculate_feature_risk_scores(correlation_results):
    """Calculate average risk scores for each feature across all protected features, This is necessary to accurately adjust the weight of each feature for model training"""

    all_risk_scores = {}
    feature_counts = {}
    average_risk_scores = {}

    for category, subcategories in correlation_results.items():
        for subcategory, df in subcategories.items():
            for _, row in df.iterrows():
                feature = row['analyzed_feature']
                risk_score = row['risk_score']

                if feature not in all_risk_scores:
                    all_risk_scores[feature] = 0
                    feature_counts[feature] = 0

                all_risk_scores[feature] += risk_score
                feature_counts[feature] += 1

    for feature, score in all_risk_scores.items():
        average_risk_scores[feature] = score / feature_counts[feature]


    return average_risk_scores

def calculate_weights_linear(risk_scores, max_reduction=0.8):
    weights = {}
    max_risk = max(risk_scores.values())

    for feature, risk in risk_scores.items():
        # Normalize risk to 0-1 scale and apply linear reduction
        normalized_risk = risk / max_risk
        weights[feature] = 1 - (normalized_risk * max_reduction)

    return weights

def apply_bias_reduction(X, y, feature_weights):
    # First, apply weights only to continuous features
    X_weighted = X.copy()

   #Apply feature weights to all rows
    for feature, weight in feature_weights.items():
        unique_values = set(X_weighted[feature].unique())
        is_binary = unique_values.issubset({0, 1, 0.0, 1.0})

        if not is_binary:
            X_weighted[feature] = X_weighted[feature] * weight


    # Now resample to balance binary protected features
    combined_data = pd.concat([X_weighted, y], axis=1)
    balanced_data = combined_data.copy()

    for feature in X.columns:
        if set(X[feature].unique()).issubset({0, 1, 0.0, 1.0}):
            # For each binary feature, balance the dataset
            group_0 = combined_data[combined_data[feature] == 0]
            group_1 = combined_data[combined_data[feature] == 1]

            # Resample the minority group to match the majority
            if len(group_0) > 0 and len(group_1) > 0:
                if len(group_0) > len(group_1):
                    group_1_resampled = resample(group_1,
                                               replace=True,
                                               n_samples=len(group_0),
                                               random_state=42)
                    balanced_data = pd.concat([group_0, group_1_resampled])
                else:
                    group_0_resampled = resample(group_0,
                                               replace=True,
                                               n_samples=len(group_1),
                                               random_state=42)
                    balanced_data = pd.concat([group_0_resampled, group_1])
                print(f"Resampled binary feature: {feature} (0s: {len(group_0)}, 1s: {len(group_1)})")
            else:
                print(f"Skipping feature {feature} - one class has no samples")

    # Split back into features and target
    X_balanced = balanced_data.drop(y.name, axis=1)
    y_balanced = balanced_data[y.name]

    return X_balanced, y_balanced

In [37]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Ensure target has no NaN values and is discrete
y_train = y_train.dropna()
X_train = X_train.loc[y_train.index]

y_test = y_test.dropna()
X_test = X_test.loc[y_test.index]

# Ensure target is properly formatted for classification
y_train = y_train.astype(int)  # Convert to integers if necessary
y_test = y_test.astype(int)

# Continue with bias reduction
risk_scores = calculate_feature_risk_scores(display_correlations)
feature_weights = calculate_weights_linear(risk_scores)

X_weighted_train, y_weighted_train = apply_bias_reduction(X_train, y_train, feature_weights)
X_weighted_test, y_weighted_test = apply_bias_reduction(X_test, y_test, feature_weights)


Resampled binary feature: adres_recentst_onderdeel_rdam (0s: 129, 1s: 2484)
Resampled binary feature: adres_recentste_buurt_groot_ijsselmonde (0s: 2604, 1s: 9)
Resampled binary feature: adres_recentste_buurt_nieuwe_westen (0s: 2603, 1s: 10)
Resampled binary feature: adres_recentste_buurt_other (0s: 1355, 1s: 1258)
Resampled binary feature: adres_recentste_buurt_oude_noorden (0s: 2607, 1s: 6)
Resampled binary feature: adres_recentste_buurt_vreewijk (0s: 2604, 1s: 9)
Resampled binary feature: adres_recentste_plaats_other (0s: 2592, 1s: 21)
Resampled binary feature: adres_recentste_plaats_rotterdam (0s: 234, 1s: 2379)
Resampled binary feature: adres_recentste_wijk_charlois (0s: 2347, 1s: 266)
Resampled binary feature: adres_recentste_wijk_delfshaven (0s: 2276, 1s: 337)
Resampled binary feature: adres_recentste_wijk_feijenoord (0s: 2159, 1s: 454)
Resampled binary feature: adres_recentste_wijk_ijsselmonde (0s: 2474, 1s: 139)
Resampled binary feature: adres_recentste_wijk_kralingen_c (0s: 24

In [38]:
print(X_train.isnull().sum())
print(y_train.isnull().sum())

adres_aantal_brp_adres                      0
adres_aantal_verschillende_wijken           0
adres_aantal_verzendadres                   0
adres_aantal_woonadres_handmatig            0
adres_dagen_op_adres                        0
                                           ..
typering_ind                                0
typering_indicatie_geheime_gegevens         0
typering_other                              0
typering_transport__logistiek___tuinbouw    0
typering_zorg__schoonmaak___welzijn         0
Length: 315, dtype: int64
0


In [39]:
rf_base_model = RandomForestClassifier(
  n_estimators=100, random_state=42, class_weight= 'balanced'
)

rf_base_model.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', random_state=42)

In [40]:
rf_weighted_model = RandomForestClassifier(
    n_estimators=100, random_state=42, class_weight='balanced'
)

rf_weighted_model.fit(X_weighted_train, y_weighted_train)

RandomForestClassifier(class_weight='balanced', random_state=42)

In [41]:
def evaluate_group_discrimination(y_true, y_pred, group_mask, group_name):
    #Evaluate discrimination metrics for a specific group
    #Accuracy for this group
    group_mask = group_mask.squeeze()
    group_acc = accuracy_score(y_true[group_mask], y_pred[group_mask])
    #Positive prediction rate for this group
    group_pos_rate = np.mean(y_pred[group_mask] == 1)

    return {
        f'{group_name}_accuracy': group_acc,
        f'{group_name}_pos_rate': group_pos_rate
    }

def evaluate_gender_bias(y_true, y_pred, X, protected_indices):
    #Evaluate gender-based discrimination
    gender_idx = protected_indices['demographic']['gender']
    gender_col = X.iloc[:, gender_idx]

    # metrics for each gender
    female_metrics = evaluate_group_discrimination(
        y_true, y_pred, gender_col == 1, 'female'
    )
    male_metrics = evaluate_group_discrimination(
        y_true, y_pred, gender_col == 0, 'male'
    )

    #calculate differences
    metrics = {
        **female_metrics,
        **male_metrics,
        'accuracy_diff_FM': female_metrics['female_accuracy'] - male_metrics['male_accuracy'],
        'pos_rate_diff_FM': female_metrics['female_pos_rate'] - male_metrics['male_pos_rate']
    }

    return metrics

def evaluate_age_bias(y_true, y_pred, X, protected_indices):
    #Evaluate age-based discrimination
    age_idx = protected_indices['demographic']['age']
    age_col = X.iloc[:, age_idx].squeeze()

    # Create age groups
    age_groups = pd.qcut(age_col, q=4, labels=['Young', 'Young Adult', 'Adult', 'Senior'])

    metrics = {}
    # Get metrics for each age group
    for age_group in age_groups.unique():
        group_metrics = evaluate_group_discrimination(
            y_true, y_pred, age_groups == age_group, f'age_{age_group.lower()}'
        )
        metrics.update(group_metrics)

    return metrics

def evaluate_location_bias(y_true, y_pred, X, protected_indices):
    #Evaluate location-based discrimination
    location_idx = protected_indices['location']['district']
    metrics = {}

    for idx in location_idx:
        district_col = X.iloc[:, idx]
        district_name = X.columns[idx].replace('address_latest_district_', '')

        district_metrics = evaluate_group_discrimination(
            y_true, y_pred, district_col == 1, f'district_{district_name}'
        )
        metrics.update(district_metrics)

    return metrics

In [42]:
def evaluate_model(model, X, y, protected_indices, model_name):
    model_pred = model.predict(X)

    report = classification_report(y, model_pred)
    accuracy = accuracy_score(y, model_pred)

    # Get discrimination metrics
    gender_metrics = evaluate_gender_bias(y, model_pred, X, protected_indices)
    age_metrics = evaluate_age_bias(y, model_pred, X, protected_indices)
    location_metrics = evaluate_location_bias(y, model_pred, X, protected_indices)

    print(f"\n--- {model_name} Set Evaluation ---")
    print(f"Overall Accuracy: {accuracy:.3f}")
    print("\nClassification Report:")
    print(report)

    print("\nGender Discrimination:")
    print(f"Female Accuracy: {gender_metrics['female_accuracy']:.3f}")
    print(f"Male Accuracy: {gender_metrics['male_accuracy']:.3f}")
    print(f"Accuracy Difference (F-M): {gender_metrics['accuracy_diff_FM']:.3f}")
    print(f"\nPositive Prediction Rates:")
    print(f"Female: {gender_metrics['female_pos_rate']:.3f}")
    print(f"Male: {gender_metrics['male_pos_rate']:.3f}")
    print(f"Difference (F-M): {gender_metrics['pos_rate_diff_FM']:.3f}")

    print("\nAge Group Discrimination:")
    for key, value in age_metrics.items():
        if 'accuracy' in key:
            print(f"{key}: {value:.3f}")

    print("\nLocation Discrimination:")
    for key, value in location_metrics.items():
        if 'pos_rate' in key:
            print(f"{key}: {value:.3f}")

    return {
        'accuracy': accuracy,
        'gender_metrics': gender_metrics,
        'age_metrics': age_metrics,
        'location_metrics': location_metrics
    }

In [43]:
protected_indices = get_protected_feature_indices()
baseline_test_eval = evaluate_model(rf_base_model, X_test, y_test, protected_indices, 'baseline Test')
baseline_train_eval = evaluate_model(rf_base_model, X_train, y_train, protected_indices,'baseline Train')
weighted_test_eval = evaluate_model(rf_weighted_model, X_weighted_test, y_weighted_test, protected_indices,'weighted Test')
weighted_train_eval = evaluate_model(rf_weighted_model, X_weighted_train, y_weighted_train, protected_indices,'weighted Train')


--- baseline Test Set Evaluation ---
Overall Accuracy: 0.853

Classification Report:
              precision    recall  f1-score   support

           0       0.85      1.00      0.92       557
           1       1.00      0.01      0.02        97

    accuracy                           0.85       654
   macro avg       0.93      0.51      0.47       654
weighted avg       0.87      0.85      0.79       654


Gender Discrimination:
Female Accuracy: 0.865
Male Accuracy: 0.841
Accuracy Difference (F-M): 0.025

Positive Prediction Rates:
Female: 0.000
Male: 0.003
Difference (F-M): -0.003

Age Group Discrimination:
age_senior_accuracy: 0.894
age_adult_accuracy: 0.915
age_young adult_accuracy: 0.841
age_young_accuracy: 0.768

Location Discrimination:
district_adres_recentste_wijk_charlois_pos_rate: 0.000
district_adres_recentste_wijk_delfshaven_pos_rate: 0.000
district_adres_recentste_wijk_feijenoord_pos_rate: 0.008
district_adres_recentste_wijk_ijsselmonde_pos_rate: 0.000
district_adres_r